# Ukázka Reinforcement Learning v knihovně Gymnasium
- open source knihovna pro vývoj algoritmů reinforcement learning
- poskytuje API pro komunikaci mezi algoritmy učení a a různými prostředími
- jedná se o fork knihovny Gym (která už není spravovaná) od vývojářů z OpenAI
- [Odkaz na knihovnu](https://gymnasium.farama.org)

In [ ]:
# Nainstaluje se do Python prostředí Jupyter notebooku
# když odkomentujete následující řádky a spustíte kód, tak se knihovny nainstalují automaticky

# %pip install gymnasium
# %pip install swig
# %pip install gymnasium[box2d]

In [40]:
import gymnasium as gym

- můžete si vybrat ze široké škály prostředí.
    - prostředí lze také modifikovat různými parametry
- v knihovně můžete najít klasických 2D prostředí kde se snažíte, aby formule nenabourala, nebo různě pohybující se 3D kostry člověka, a nebo prostředí, kde lze naučit agenta hrát blackjack
- pokud vám, žádné z nich nevyhovuje, tak lze nainportovat externí prostředí (třeba ze hry TrackMania)

In [3]:
# Výpis všech dostupných prostředí
gym.pprint_registry()

===== classic_control =====
Acrobot-v1             CartPole-v0            CartPole-v1
MountainCar-v0         MountainCarContinuous-v0 Pendulum-v1
===== phys2d =====
phys2d/CartPole-v0     phys2d/CartPole-v1     phys2d/Pendulum-v0
===== box2d =====
BipedalWalker-v3       BipedalWalkerHardcore-v3 CarRacing-v3
LunarLander-v3         LunarLanderContinuous-v3
===== toy_text =====
Blackjack-v1           CliffWalking-v0        FrozenLake-v1
FrozenLake8x8-v1       Taxi-v3
===== tabular =====
tabular/Blackjack-v0   tabular/CliffWalking-v0
===== mujoco =====
Ant-v2                 Ant-v3                 Ant-v4
Ant-v5                 HalfCheetah-v2         HalfCheetah-v3
HalfCheetah-v4         HalfCheetah-v5         Hopper-v2
Hopper-v3              Hopper-v4              Hopper-v5
Humanoid-v2            Humanoid-v3            Humanoid-v4
Humanoid-v5            HumanoidStandup-v2     HumanoidStandup-v4
HumanoidStandup-v5     InvertedDoublePendulum-v2 InvertedDoublePendulum-v4
InvertedDoublePendulu

## Přistání na měsíci
- my si vybereme prostředí [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/), kde je cílem, aby lunární modul bezpečně přistál na "měsíci"
- jedná se o problém optimalizace trajektorie rakety
- náš modul má diskrétní prostor akcí ze kterých může vybírat
    - 0: nedělat nic
    - 1: spustí levý balanční motor
    - 2: spustí hlavní motor
    - 3: spustí pravý balanční motor
- **observation** je reprezentován jako osmirozměrný vektor: 
    - souřadnice modulu v x a y
    - jeho lineární rychlosti v x a y
    - jeho úhel naklonění a úhlová rychlost
    - dva booleany, které vyjadřují, zda je každá noha v kontaktu se zemí, nebo ne
- chceme **agenta** naučit bezpečnému přistání modulu
    - cílem je získat dostatečný počet bodů (cíl je 200 bodů), které lze získat za různé úkoly v každém kroku
    - třeba přistátní modulu je hodnoceno 100 body, naopak -100 body havarování
    - ale i rychlost pohybu, či výška modulu je hodnocena
- informaci o prostředí je spousta, takže spíš si doporučuji přečíst dokumentaci :)
- k vytvoření ukázky jsem využil následující [návod](https://www.geeksforgeeks.org/actor-critic-algorithm-in-reinforcement-learning/)

In [46]:
# Nejprve si ukážeme jak to vůbec funguje
# Ukážeme si jeden běh epizody
# Epizoda je běh modelu v nějakém prostředí (konec epizody může být například havarie modulu nebo jeho přistání)
# Během učení může agent projít mnoha epizodami

# Inicializace prostředí přistání na měsíci
env = gym.make("LunarLander-v3", render_mode="human")
# Hodnota gym.Env vrácená funkcí gym.make je třída reprezentující Markovův rozhodovací proces
# Více: https://gymnasium.farama.org/api/env/#gymnasium.Env

# Zahájíme novou epizodu
observation, info = env.reset(seed=42)
episode_over = False
# Iterace časem v dané epizodě
while not episode_over:
    # Zde v aktuálním případě bereme náhodnou akci z prostoru akcí
    action = env.action_space.sample()
    
    # Funkce step provede vybranou akci v prostředí
    # výsledkem je observation, reward a informaci o tom zda epizoda byla terminated (například v případě havárie) nebo truncaded (když chceme pevný počet časových kroků)
    observation, reward, terminated, truncated, info = env.step(action)

    # Pokud nějakým způsobem epizoda skončila, tak zahájíme novou (restartujeme prostředí)
    if terminated or truncated:
        episode_over = True

env.close()
# Pozn. Pokud se pokusíte zavřít okno, tak se vám může stát, že vám spadne celý Jupyter kernel.
# Doporučuji nehcávat okno otevřené na pozadí. 

In [47]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
import tensorflow as tf

import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.18.0
Keras version: 3.6.0


## Actor-Critic algoritmus
- kombinuje policy-based a value-based přístupy
- skládá se ze dvou konceptů
    - Actor (herec) – reprezentuje politiku, tedy stárá se o to, jak agent jedná
    - Critic (kritik) – poskytuje zpětnou vazbu herci tím, že hodnotí jeho akce

In [ ]:
from keras import layers

env = gym.make("LunarLander-v3", render_mode="human")

# Síť nám bude předkládat akce, kde akce s největší pravděpodobností by měla maximalizovat šanci na úspěch
actor = keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(env.action_space.n, activation='softmax')
])

critic = keras.Sequential([
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

actor_optimizer = keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Training loop
num_episodes = 100
gamma = 0.99

for i in range(num_episodes):
    observation, info = env.reset(seed=42)
    episode_reward = 0

    with tf.GradientTape(persistent=True) as tape:
        # Omezení počtu iterací v epizodě
        for t in range(1, 10000):  
            # Vypočítáme pravděpodobnost akcí
            action_probs = actor(np.array([observation]))
            # Výpočet očekávané hodnoty odměny
            state_value = critic(np.array([observation]))[0, 0]
            # Výběr z akcí náhodně, ale s přihlédnutím na vypočítanou pravděpodobnost akcí
            action = np.random.choice(env.action_space.n, p=action_probs.numpy()[0])

            # Provedeme vybranou akci
            observation, reward, terminated, truncated, info = env.step(action)
            episode_reward += reward

            # Výpočet aktuální hodnoty odměny
            next_state_value = critic(np.array([observation]))[0, 0]
            # Temporal Difference – chyba měří, rozdíl mezi očekávanou a aktuální hodnotou odměny
            td = reward + gamma * next_state_value - state_value

            # Vypočítáme ztráty
            actor_loss = -tf.math.log(action_probs[0, action]) * td
            critic_loss = tf.square(td)

            # Aktualizujeme váhy modelů na základě ztrát
            actor_gradients = tape.gradient(actor_loss, actor.trainable_variables)
            critic_gradients = tape.gradient(critic_loss, critic.trainable_variables)
            actor_optimizer.apply_gradients(zip(actor_gradients, actor.trainable_variables))
            critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))

            if terminated or truncated or abs(reward) == 100:
                break

    if i % 10 == 0:
        print(f"Episode {i}, Reward: {episode_reward}")

env.close()